In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x # Ensuring that TensorFlow 2.x is being used
except Exception:
  pass

import tensorflow as tf

tf.compat.v1.enable_eager_execution()

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x # Ensuring that TensorFlow 2.x is being used`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
!pip install transformers

In [0]:
import numpy as np
import pandas as pd
import warnings
import tensorflow.keras as keras
warnings.filterwarnings('ignore')

from transformers import (
    BertConfig,
    BertTokenizer,
    TFBertModel,
    TFDistilBertModel,
    DistilBertTokenizer
)

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)

In [5]:
df.head()

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1


In [9]:
batch_1 = df[:5000]
batch_1[1].value_counts()

1    2607
0    2393
Name: 1, dtype: int64

In [0]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [11]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 73)

In [12]:
input_layer = keras.layers.Input(shape = (attention_mask.shape[1],), dtype='int64')
bert = TFBertModel.from_pretrained("bert-base-cased")(input_layer)
bert = bert[0][:,0,:]              
classifier = keras.layers.Dense(units=1, activation='sigmoid')(bert)
model = keras.models.Model(inputs=input_layer, outputs=classifier)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 73)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 73, 768), (None,  108310272 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 108,311,041
Trainable params: 108,311,041
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics = ["accuracy"])

In [14]:
model.fit(
    [padded, attention_mask],
    [np.array(batch_1[1])],
    epochs=8, 
    validation_split=0.1)

Train on 4500 samples, validate on 500 samples
Epoch 1/8
4500/4500 [==============================] - 62s 14ms/sample - loss: 0.4533 - accuracy: 0.7600 - val_loss: 0.3727 - val_accuracy: 0.8420
Epoch 2/8
4500/4500 [==============================] - 47s 10ms/sample - loss: 0.1851 - accuracy: 0.9311 - val_loss: 0.3512 - val_accuracy: 0.8640
Epoch 3/8
4500/4500 [==============================] - 47s 10ms/sample - loss: 0.0539 - accuracy: 0.9813 - val_loss: 0.4300 - val_accuracy: 0.8720
Epoch 4/8
4500/4500 [==============================] - 47s 11ms/sample - loss: 0.0487 - accuracy: 0.9833 - val_loss: 0.4123 - val_accuracy: 0.8760
Epoch 5/8
4500/4500 [==============================] - 47s 11ms/sample - loss: 0.0298 - accuracy: 0.9904 - val_loss: 0.5679 - val_accuracy: 0.8700
Epoch 6/8
4500/4500 [==============================] - 47s 11ms/sample - loss: 0.0200 - accuracy: 0.9940 - val_loss: 0.5705 - val_accuracy: 0.8740
Epoch 7/8
4500/4500 [==============================] - 47s 11ms/sample 

# Distil Bert Model

In [0]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [17]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 73)

In [18]:
input_layer = keras.layers.Input(shape = (attention_mask.shape[1],), dtype='int64')
bert = TFDistilBertModel.from_pretrained("distilbert-base-cased")(input_layer)
bert = bert[0][:,0,:]              
classifier = keras.layers.Dense(units=1, activation='sigmoid')(bert)
model = keras.models.Model(inputs=input_layer, outputs=classifier)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 73)]              0         
_________________________________________________________________
tf_distil_bert_model (TFDist ((None, 73, 768),)        65190912  
_________________________________________________________________
tf_op_layer_strided_slice_1  [(None, 768)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 769       
Total params: 65,191,681
Trainable params: 65,191,681
Non-trainable params: 0
_________________________________________________________________


In [0]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
model.compile(optimizer=opt, loss="binary_crossentropy", metrics = ["accuracy"])

In [21]:
model.fit(
    [padded, attention_mask],
    [np.array(batch_1[1])],
    epochs=10, 
    validation_split=0.1)

Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 26s 6ms/sample - loss: 0.4504 - accuracy: 0.7700 - val_loss: 0.3257 - val_accuracy: 0.8640
Epoch 2/10
4500/4500 [==============================] - 25s 5ms/sample - loss: 0.2193 - accuracy: 0.9151 - val_loss: 0.3053 - val_accuracy: 0.8780
Epoch 3/10
4500/4500 [==============================] - 25s 5ms/sample - loss: 0.0866 - accuracy: 0.9709 - val_loss: 0.3380 - val_accuracy: 0.9020
Epoch 4/10
4500/4500 [==============================] - 25s 5ms/sample - loss: 0.0467 - accuracy: 0.9840 - val_loss: 0.4044 - val_accuracy: 0.8960
Epoch 5/10
4500/4500 [==============================] - 24s 5ms/sample - loss: 0.0297 - accuracy: 0.9893 - val_loss: 0.5301 - val_accuracy: 0.8720
Epoch 6/10
4500/4500 [==============================] - 24s 5ms/sample - loss: 0.0248 - accuracy: 0.9900 - val_loss: 0.6800 - val_accuracy: 0.8380
Epoch 7/10
4500/4500 [==============================] - 24s 5ms/sample 